<center><h1>WEEK 3 ASSIGNMENT: Clustering Toronto Neighborhoods<h1/><center> <hr>

Le'ts start by importing the libraries needed

In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json 

!pip install geopy
from geopy.geocoders import Nominatim 

import requests 

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

from bs4 import BeautifulSoup

!pip install folium
import folium 

print('Libraries imported')

Libraries imported


<h2> SCRAPING WIKIPEDIA </h2>

The Scraping Wikipedia part of this code follows the following GitHub script: https://github.com/Sateesh110/Rep_Medium/blob/master/A1_WikiTables_Scraping/A1_WikiTable_WorldPopulation.ipynb

First, let's make a DataFrame of the Wikipedia Table we need for our Clustering.

In [2]:
#Table with the Toronto Postal Codes, Boroughs, and Neighborhoods
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [3]:
#Making a get request
s=requests.Session()
response= s.get(url, timeout=10)
response

<Response [200]>

In [4]:
#Parsing the response content to html
soup=BeautifulSoup(response.content, 'html.parser')

In [5]:
#Viewing the content in html format
pretty_soup=soup.prettify()

In [6]:
soup.title.string

'List of postal codes of Canada: M - Wikipedia'

In [7]:
#Find all the tables in the html
all_tables=soup.find_all('table')

In [8]:
#Het the right table to scrap
right_table=soup.find('table', {'class': 'wikitable sortable'})

In [9]:
#Number of columns in the table
for row in right_table.findAll('tr'):
    cells = row.findAll('td')
    
len(cells)

3

In [10]:
#Number of rows in the table including the header
rows=right_table.findAll('tr')
len(rows)

181

In [11]:
#Header attributes of the table
header = [th.text.rstrip() for th in rows[0].find_all('th')]
print(header)
print('--------------')
print(len(header))

['Postal Code', 'Borough', 'Neighbourhood']
--------------
3


In [12]:
#Get tabular data

lst_data=[]
for row in rows[1:]:
    data = [d.text.rstrip() for d in row.find_all('td')]
    lst_data.append(data)

In [13]:
#Alternatively, select also works as find_all
lst_data1=[]
for row in rows[1:]:
    data= [d.text.rstrip() for d in row.select('td')]
    lst_data1.append(data)

In [14]:
#sample records
lst_data[0:5]

[['M1A', 'Not assigned', 'Not assigned'],
 ['M2A', 'Not assigned', 'Not assigned'],
 ['M3A', 'North York', 'Parkwoods'],
 ['M4A', 'North York', 'Victoria Village'],
 ['M5A', 'Downtown Toronto', 'Regent Park, Harbourfront']]

In [15]:
lst_data1[0:5]

[['M1A', 'Not assigned', 'Not assigned'],
 ['M2A', 'Not assigned', 'Not assigned'],
 ['M3A', 'North York', 'Parkwoods'],
 ['M4A', 'North York', 'Victoria Village'],
 ['M5A', 'Downtown Toronto', 'Regent Park, Harbourfront']]

In [16]:
#Length of each record
len(lst_data[0])

3

In [17]:
#Number of rows excluding the header
len(lst_data)

180

<h2>CREATING THE DATAFRAME<h2/>

Let's start with an empty DataFrame and append the values from our lst_data

In [18]:
#Empty DataFrame
df_toronto=pd.DataFrame(columns=header)

In [19]:
df_toronto

,Postal Code,Borough,Neighbourhood


In [20]:
#Appending cells from lst_data
for i,data in enumerate(lst_data,0):
    postal_code= lst_data[i][0]
    borough=lst_data[i][1]
    neighborhood=lst_data[i][2]
    
    df_toronto=df_toronto.append({'Postal Code':postal_code,
                                 'Borough':borough,
                                 'Neighbourhood':neighborhood}, ignore_index=True)

In [21]:
df_toronto.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [22]:
df_toronto.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180 entries, 0 to 179
Data columns (total 3 columns):
Postal Code      180 non-null object
Borough          180 non-null object
Neighbourhood    180 non-null object
dtypes: object(3)
memory usage: 4.3+ KB


In [23]:
#Drop rows with 'Not assigned' values
df_toronto.loc[(df_toronto['Borough']=='Not assigned'),:]=np.nan

#Assigning the Borough name to Neighborhoods where the name is 'Not assigned'
df_toronto.loc[(df_toronto['Neighbourhood']=='Not assigned'),'Neighbourhood']=df_toronto['Borough']

In [24]:
df_toronto=df_toronto.dropna(axis=0).reset_index(drop=True)


In [25]:
df_toronto.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [26]:
df_toronto.shape

(103, 3)

In [27]:
#Changing the name of column 3 to 'Neighborhood'
df_toronto.columns.values[2]='Neighborhood'

In [28]:
df_toronto

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [29]:
#Creating a new table with the Latitude and Longitude
latlon=['Latitude', 'Longitude']
df_latlon= pd.DataFrame(columns=latlon)
df_latlon

,Latitude,Longitude


Code for getting the Latitude and Longitude values through the geocoder module and filling in the DataFrame

We will import it as csv from an already existing table. 

In [30]:
'''
!pip install geocoder

import geocoder # import geocoder

for postal_code in df_toronto['Postal Code']:
    # initialize your variable to None
    lat_lng_coords = None

    # loop until you get the coordinates
    while(lat_lng_coords is None):
      g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
      lat_lng_coords = g.latlng

    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    
    df_latlon=df_latlon.append({'Latitude':latitude,
                               'Longitude':longitude}, ignore_index=True)
'''

"\n!pip install geocoder\n\nimport geocoder # import geocoder\n\nfor postal_code in df_toronto['Postal Code']:\n    # initialize your variable to None\n    lat_lng_coords = None\n\n    # loop until you get the coordinates\n    while(lat_lng_coords is None):\n      g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))\n      lat_lng_coords = g.latlng\n\n    latitude = lat_lng_coords[0]\n    longitude = lat_lng_coords[1]\n    \n    df_latlon=df_latlon.append({'Latitude':latitude,\n                               'Longitude':longitude}, ignore_index=True)\n"

Creating a table with the geospatial coordinates from our CSV file. 

In [63]:
df_coordinates= pd.read_csv('Geospatial_Coordinates (1).csv')

In [64]:
df_coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Joining the two tables on Postal Code.

In [66]:
df_toronto=df_toronto.join(df_coordinates.set_index('Postal Code'), on='Postal Code')

In [67]:
df_toronto

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


We will only Cluster neighborhoods in Toronto (whose Borough name includes Toronto)

In [68]:
#Dropping rows with places outside Toronto
df_t= df_toronto.loc[(name.split(' ')[-1]=='Toronto' for name in df_toronto['Borough']), :].reset_index(drop=True)

In [69]:
df_t

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031
5,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
6,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
7,M6G,Downtown Toronto,Christie,43.669542,-79.422564
8,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568
9,M6H,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259


<h2>Now we are going to start using the Foursquare API to explore the neighborhoods and segment them<h2/>

In [70]:
#We first define Foursquare Credentials and Version, the following cell is hidden

In [71]:
#Defining Foursquare Credentials and Version


CLIENT_ID= '2JN5Y5PQPURYGVO4TVIUJJTXSC11CZLQL5A2MCKURZ0HS0GZ'
CLIENT_SECRET= '5TEQ53TGFISDCT0QBEOM0SSDDXZISBTECEXBTWJHRNH4UVAF'
VERSION='20201122'
LIMIT=100

print('Your credentials were entered')
#print('Client_ID:',CLIENT_ID)
#print('Client_Secret:',CLIENT_SECRET)

Your credentials were entered


<br>
<br>
Defining a function that returns a maximum of 100 popular venues in each of the Toronto Neighborhood

In [73]:
#Function loops through each neighborhood on our list one by one, and isolates the relevant information in the JSON file returned by the get request. 

def getNearbyVenues (names, latitudes, longitudes, radius=500):
    venues_list=[]
    for name, lat, lng, in zip (names, latitudes, longitudes):
        print(name)
        
        #Create the API request URL
        url='https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
        CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, LIMIT)
        
        #Make the Get Request
        results=requests.get(url).json()['response']['groups'][0]['items']
        
        #Now, let's collect the relevant information for each nearby venue
        venues_list.append([(
            name,
            lat,
            lng,
            v['venue']['name'],
            v['venue']['location']['lat'],
            v['venue']['location']['lng'],
            v['venue']['categories'][0]['name']) for v in results])
            
    nearby_venues= pd.DataFrame( item for venue_list in venues_list for item in venue_list)
        
    nearby_venues.columns=['Neighborhood','Neighborhood Latitude','Neighborhood Longitude','Venue','Venue Latitude','Venue Longitude','Venue Category']
            
    return (nearby_venues)

<br>
<br>
Now we run the above function on each neighborhood in order to create a new DataFrame called toronto_venues

In [74]:
toronto_venues= getNearbyVenues(names=df_t['Neighborhood'], 
                               latitudes=df_t['Latitude'],
                               longitudes=df_t['Longitude'])

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West,  Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport


<br>
<br>
We have a total of 1639 venues in our DataFrame

In [75]:
toronto_venues.shape

(1639, 7)

In [76]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant
4,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa


**Let's check the number of venues in each neighborhood**

In [77]:
toronto_venues[['Neighborhood','Venue Category']].groupby(['Neighborhood']).count().rename(columns={'Venue Category':'Venues Count'})

,Venues Count
Neighborhood,
Berczy Park,55
"Brockton, Parkdale Village, Exhibition Place",25
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",16
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",17
Central Bay Street,68
Christie,16
Church and Wellesley,75
"Commerce Court, Victoria Hotel",100
Davisville,34


<br>
<br>
Now, let's explore the number of unique categories among all venues

In [78]:
print('There are {} unique categories'.format(len(set(toronto_venues['Venue Category']))))

There are 240 unique categories


<h2> ANALYZING EACH NEIGHBORHOOD <h2/>

In [79]:
#One Hot Encoding
toronto_onehot=pd.get_dummies(toronto_venues[['Venue Category']], prefix=' ',prefix_sep=' ')

#Add the Neighborhood column to the DataFrame
toronto_onehot['Neighborhood']=toronto_venues['Neighborhood']

#Let's make it the first column
fixed_columns=[toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot=toronto_onehot[fixed_columns]
toronto_onehot.head()
                

,Neighborhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sri Lankan Restaurant,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Reg

<br>
<br>
Let's examine the size of our new dataframe

In [80]:
toronto_onehot.shape

(1639, 241)

We have one row for each venue

<h3>Now, we are ready to group the neighborhoods by row and calculate the mean frequencies per Venue Category<h3/>

In [81]:
toronto_grouped=toronto_onehot.groupby(['Neighborhood']).mean().reset_index()
toronto_grouped

,Neighborhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sri Lankan Restaurant,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.018182,0.000000,0.000000,0.000000,0.0000,0.0000,0.000000,0.0000,0.018182,0.036364,0.000000,0.000000,0.00,0.018182,0.018182,0.000000,0.036364,0.000000,0.000000,0.018182,0.000000,0.000000,0.000000,0.000000,0.00,0.018182,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.0000,0.018182,0.000000,0.000000,0.000000,0.000000,0.036364,0.000000,0.000000,0.000000,0.00,0.000000,0.036364,0.090909,0.000000,0.000000,0.000000

<br>
<br>
Let's extract the top 15 venues for each neighborhood
<br>
<br>
We first define a function to sort the venues in descending order

In [82]:
def return_most_common_venues(row, num_top_venues):
    row_categories=row.iloc[1:] #We drop the Neighborhood column for sorting
    row_categories_sorted=row_categories.sort_values(ascending=False) #We have a Series with the Venue Categories as Indices and order those by frequency 
    
    return row_categories_sorted.index.values[0: num_top_venues] 

Now we can create a new DataFrame to display the top 15 venues for each neighborhood

In [83]:
num_top_venues= 15
indicators=['st','nd','rd']

#Creating the Columns
columns=['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

#Creating a new DataFrame
#Setting the columns
neighborhood_venues_sorted=pd.DataFrame(columns=columns)
#Filling in the Neighborhood column
neighborhood_venues_sorted['Neighborhood']=toronto_grouped['Neighborhood']
#Filling in the venue columnns; looping through all rows
for ind in np.arange(toronto_grouped.shape[0]): 
    #Locating column and populating columns [1:-1]
    neighborhood_venues_sorted.iloc[ind,1:] = return_most_common_venues(toronto_grouped.iloc[ind,:], num_top_venues)

neighborhood_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
0,Berczy Park,Coffee Shop,Bakery,Cocktail Bar,Farmers Market,Beer Bar,Restaurant,Cheese Shop,Seafood Restaurant,Concert Hall,Basketball Stadium,Beach,Hotel,Shopping Mall,Bistro,Department Store
1,"Brockton, Parkdale Village, Exhibition Place",Café,Breakfast Spot,Nightclub,Coffee Shop,Gym,Playground,Stadium,Bar,Italian Restaurant,Bakery,Restaurant,Climbing Gym,Bookstore,Intersection,Furniture / Home Store
2,"Business reply mail Processing Centre, South C...",Light Rail Station,Pizza Place,Brewery,Burrito Place,Butcher,Farmers Market,Fast Food Restaurant,Restaurant,Recording Studio,Auto Workshop,Skate Park,Garden,Garden Center,Comic Shop,Gym / Fitness Center
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Lounge,Airport Service,Boutique,Bar,Boat or Ferry,Sculpture Garden,Historic Site,Rental Car Location,Harbor / Marina,Coffee Shop,Plane,Airport Terminal,Airport Gate,Airport Food Court,Airport
4,Central Bay Street,Coffee Shop,Café,Sandwich Place,Italian Restaurant,Department Store,Japanese Restaurant,Salad Place,Bubble Tea Shop,Burger Joint,Thai Restaurant,Modern European Restaurant,New American Restaurant,Park,Poke Place,Middle Eastern Restaurant


<br>
<br>
<h2> CLUSTERING NEIGHBORHOODS USING KMEANS

We run the KMeans clustering algorithm from scikit learn to group Toronto's Neighborhoods into 8 distinct clusters

In [84]:
# Setting the number of clusters
Kclusters=8

# Dropping the Neighborhood column for the clustering 
toronto_grouped_clustering=toronto_grouped.drop('Neighborhood', axis=1)

# KMeans Clustering
k_means=KMeans(n_clusters=Kclusters, random_state=0).fit(toronto_grouped_clustering)

#Checking Cluster labels generated for each row in the DataFrame
k_means.labels_[0:40]

array([7, 7, 0, 0, 7, 0, 7, 7, 0, 0, 0, 7, 5, 7, 7, 0, 0, 0, 4, 7, 2, 7,
       0, 7, 7, 7, 6, 1, 7, 7, 0, 7, 7, 7, 7, 3, 0, 7, 7], dtype=int32)

Let's create a new DataFrame that contains the cluster as well as the top 10 venues for each neighborhood

In [85]:
#Adding Cluster Labels
neighborhood_venues_sorted.insert(0,'Cluster Labels', k_means.labels_)

toronto_merged=df_t

#Merging neighborhood_venues_sorted with df_t to add latitude/longitude for each neighborhood 

toronto_merged=df_t.join(neighborhood_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,7,Coffee Shop,Park,Bakery,Pub,Breakfast Spot,Café,Theater,Shoe Store,Brewery,Restaurant,Chocolate Shop,Performing Arts Venue,Mexican Restaurant,Spa,Dessert Shop
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,7,Coffee Shop,Yoga Studio,Diner,Restaurant,Portuguese Restaurant,Park,Music Venue,Mexican Restaurant,Italian Restaurant,Hobby Shop,Fried Chicken Joint,Distribution Center,Creperie,Smoothie Shop,College Cafeteria
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,7,Coffee Shop,Clothing Store,Café,Bubble Tea Shop,Japanese Restaurant,Cosmetics Shop,Hotel,Bookstore,Pizza Place,Middle Eastern Restaurant,Lingerie Store,Italian Restaurant,Ramen Restaurant,Furniture / Home Store,Theater
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,7,Coffee Shop,Café,Cocktail Bar,Restaurant,Beer Bar,Gastropub,American Restaurant,Farmers Market,Hotel,Japanese Restaurant,Lingerie Store,Department Store,Cosmetics Shop,Moroccan Restaurant,Clothing Store
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,3,Pub,Health Food Store,Trail,Neighborhood,Yoga Studio,Dive Bar,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Department Store,Doner Restaurant,Donut Shop,Dumpling Restaurant
5,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,7,Coffee Shop,Bakery,Cocktail Bar,Farmers Market,Beer Bar,Restaurant,Cheese Shop,Seafood Restaurant,Concert Hall,Basketball Stadium,Beach,Hotel,Shopping Mall,Bistro,Department Store
6,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,7,Coffee Shop,Café,Sandwich Place,Italian Restaurant,Department Store,Japanese Restaurant,Salad Place,Bubble Tea Shop,Burger Joint,Thai Restaurant,Modern European Restaurant,New American Restaurant,Park,Poke Place,Middle Eastern Restaurant
7,M6G,Downtown Toronto,Christie,43.669542,-79.422564,0,Grocery Store,Café,Park,Restaurant,Candy Store,Athletics & Sports,Italian Restaurant,Baby Store,Coffee Shop,Nightclub,Doner Restaurant,Dive Bar,Distribution Center,Donut Shop,Discount Store
8,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568,7,Coffee Shop,Café,Gym,Restaurant,Hotel,Clothing Store,Thai Restaurant,Bar,Sushi Restaurant,Bookstore,Salad Place,Pizza Place,Office,Concert Hall,Cosmetics Shop
9,M6H,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259,0,Pharmacy,Bakery,Middle Eastern Restaurant,Bank,Bar,Café,Music Venue,Supermarket,Brewery,Park,Grocery Store,Donut Shop,Dumpling Restaurant,Diner,Dog Run


**Finally, we can visualize the results of our Clustering on the Map**

In [86]:
#Let's find the Coordinates for Toronto

address='Toronto, Ontario'
geolocator=Nominatim(user_agent='toronto_explorer')
location=geolocator.geocode(address)
latitude=location.latitude
longitude=location.longitude

#Create Toronto Map

map_clusters=folium.Map(location=[latitude,longitude], zoom_start=11)

#Setting a color scheme for the clusters
x = np.arange(Kclusters)
ys = [i + x + (i*x)**2 for i in range(Kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

#Add Markers to the map

markers_colors=[]
for lat, lng, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    
    label=folium.Popup(str(poi)+', Cluster '+str(cluster), parse_html=True)
    folium.CircleMarker([lat,lng], radius=5, popup=label, color=rainbow[cluster-1], fill=True, fill_color=rainbow[cluster-1], fill_opacity=0.7).add_to(map_clusters)
    
map_clusters

In [87]:
cluster7=toronto_merged.loc[(toronto_merged['Cluster Labels']==7),:]
cluster7=cluster7.iloc[:,6:]

venues7_list=[]


for index in range(len(cluster7.index.values)):
    for column in range(len(cluster7.columns.values)):
    
        venues7_list.append(cluster7.iloc[index,column])
venues7_list.sort()


<br>
<br>
Now, we can explore each Cluster: 

<h3>Cluster 0

In [88]:
cluster0=toronto_merged.loc[toronto_merged['Cluster Labels']==0, toronto_merged.columns[[2] + list(range(5,toronto_merged.shape[1]))]]
cluster0

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
7,Christie,0,Grocery Store,Café,Park,Restaurant,Candy Store,Athletics & Sports,Italian Restaurant,Baby Store,Coffee Shop,Nightclub,Doner Restaurant,Dive Bar,Distribution Center,Donut Shop,Discount Store
9,"Dufferin, Dovercourt Village",0,Pharmacy,Bakery,Middle Eastern Restaurant,Bank,Bar,Café,Music Venue,Supermarket,Brewery,Park,Grocery Store,Donut Shop,Dumpling Restaurant,Diner,Dog Run
12,"The Danforth West, Riverdale",0,Greek Restaurant,Coffee Shop,Italian Restaurant,Bookstore,Furniture / Home Store,Restaurant,Ice Cream Shop,Cosmetics Shop,Caribbean Restaurant,Pub,Lounge,Bakery,Liquor Store,Pizza Place,Dessert Shop
15,"India Bazaar, The Beaches West",0,Park,Gym,Pub,Brewery,Liquor Store,Sandwich Place,Burrito Place,Italian Restaurant,Restaurant,Fast Food Restaurant,Fish & Chips Shop,Ice Cream Shop,Steakhouse,Sushi Restaurant,Coffee Shop
20,Davisville North,0,Department Store,Sandwich Place,Dance Studio,Hotel,Food & Drink Shop,Dog Run,Breakfast Spot,Gym / Fitness Center,Park,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dim Sum Restaurant,Eastern European Restaurant,Dive Bar
22,"High Park, The Junction South",0,Mexican Restaurant,Bar,Thai Restaurant,Café,Arts & Crafts Store,Discount Store,Flea Market,Diner,Italian Restaurant,Bakery,Fried Chicken Joint,Fast Food Restaurant,Cajun / Creole Restaurant,Furniture / Home Store,Speakeasy
25,"Parkdale, Roncesvalles",0,Breakfast Spot,Gift Shop,Dessert Shop,Cuban Restaurant,Eastern European Restaurant,Dog Run,Bar,Italian Restaurant,Restaurant,Movie Theater,Bookstore,Coffee Shop,Cupcake Shop,Distribution Center,Electronics Store
26,Davisville,0,Dessert Shop,Sandwich Place,Pizza Place,Gym,Italian Restaurant,Café,Coffee Shop,Sushi Restaurant,Pharmacy,Seafood Restaurant,Restaurant,Diner,Farmers Market,Indoor Play Area,Indian Restaurant
30,"Kensington Market, Chinatown, Grange Park",0,Vegetarian / Vegan Restaurant,Mexican Restaurant,Bar,Coffee Shop,Café,Vietnamese Restaurant,Pizza Place,Park,Burger Joint,Dumpling Restaurant,Dessert Shop,Grocery Store,Bakery,Gaming Cafe,Snack Place
32,"CN Tower, King and Spadina, Railway Lands, Har...",0,Airport Lounge,Airport Service,Boutique,Bar,Boat or Ferry,Sculpture Garden,Historic Site,Rental Car Location,Harbor / Marina,Coffee Shop,Plane,Airport Terminal,Airport Gate,Airport Food Court,Airport


<h3>Cluster 1

In [89]:
cluster1=toronto_merged.loc[toronto_merged['Cluster Labels']==1, toronto_merged.columns[[2] + list(range(5,toronto_merged.shape[1]))]]
cluster1

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
19,Roselawn,1,Dive Bar,Garden,Music Venue,Yoga Studio,Department Store,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner,Dim Sum Restaurant


<h3>Cluster 2

In [90]:
cluster2=toronto_merged.loc[toronto_merged['Cluster Labels']==1, toronto_merged.columns[[2] + list(range(5,toronto_merged.shape[1]))]]
cluster2

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
19,Roselawn,1,Dive Bar,Garden,Music Venue,Yoga Studio,Department Store,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store,Diner,Dim Sum Restaurant


<h3> Cluster 3

In [91]:
cluster3=toronto_merged.loc[toronto_merged['Cluster Labels']==3, toronto_merged.columns[[2] + list(range(5,toronto_merged.shape[1]))]]
cluster3

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
4,The Beaches,3,Pub,Health Food Store,Trail,Neighborhood,Yoga Studio,Dive Bar,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Department Store,Doner Restaurant,Donut Shop,Dumpling Restaurant


<h3> Cluster 4

In [92]:
cluster4=toronto_merged.loc[toronto_merged['Cluster Labels']==4, toronto_merged.columns[[2] + list(range(5,toronto_merged.shape[1]))]]
cluster4

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
18,Lawrence Park,4,Park,Bus Line,Swim School,Dim Sum Restaurant,Escape Room,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Dive Bar,Distribution Center,Discount Store,Diner


<h3> Cluster 5

In [93]:
cluster5=toronto_merged.loc[toronto_merged['Cluster Labels']==5, toronto_merged.columns[[2] + list(range(5,toronto_merged.shape[1]))]]
cluster5


,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
21,"Forest Hill North & West, Forest Hill Road Park",5,Sushi Restaurant,Park,Mexican Restaurant,Jewelry Store,Trail,Dive Bar,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Yoga Studio,Dog Run,Dessert Shop,Donut Shop,Dumpling Restaurant


<h3> Cluster 6

In [94]:
cluster6=toronto_merged.loc[toronto_merged['Cluster Labels']==6, toronto_merged.columns[[2] + list(range(5,toronto_merged.shape[1]))]]
cluster6



,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
33,Rosedale,6,Park,Trail,Tennis Court,Playground,Donut Shop,Doner Restaurant,Dog Run,Dumpling Restaurant,Deli / Bodega,Dive Bar,Distribution Center,Discount Store,Eastern European Restaurant,Diner,Dim Sum Restaurant


<h3> Cluster 7

In [95]:
cluster7=toronto_merged.loc[toronto_merged['Cluster Labels']==7, toronto_merged.columns[[2] + list(range(5,toronto_merged.shape[1]))]]
cluster7



,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
0,"Regent Park, Harbourfront",7,Coffee Shop,Park,Bakery,Pub,Breakfast Spot,Café,Theater,Shoe Store,Brewery,Restaurant,Chocolate Shop,Performing Arts Venue,Mexican Restaurant,Spa,Dessert Shop
1,"Queen's Park, Ontario Provincial Government",7,Coffee Shop,Yoga Studio,Diner,Restaurant,Portuguese Restaurant,Park,Music Venue,Mexican Restaurant,Italian Restaurant,Hobby Shop,Fried Chicken Joint,Distribution Center,Creperie,Smoothie Shop,College Cafeteria
2,"Garden District, Ryerson",7,Coffee Shop,Clothing Store,Café,Bubble Tea Shop,Japanese Restaurant,Cosmetics Shop,Hotel,Bookstore,Pizza Place,Middle Eastern Restaurant,Lingerie Store,Italian Restaurant,Ramen Restaurant,Furniture / Home Store,Theater
3,St. James Town,7,Coffee Shop,Café,Cocktail Bar,Restaurant,Beer Bar,Gastropub,American Restaurant,Farmers Market,Hotel,Japanese Restaurant,Lingerie Store,Department Store,Cosmetics Shop,Moroccan Restaurant,Clothing Store
5,Berczy Park,7,Coffee Shop,Bakery,Cocktail Bar,Farmers Market,Beer Bar,Restaurant,Cheese Shop,Seafood Restaurant,Concert Hall,Basketball Stadium,Beach,Hotel,Shopping Mall,Bistro,Department Store
6,Central Bay Street,7,Coffee Shop,Café,Sandwich Place,Italian Restaurant,Department Store,Japanese Restaurant,Salad Place,Bubble Tea Shop,Burger Joint,Thai Restaurant,Modern European Restaurant,New American Restaurant,Park,Poke Place,Middle Eastern Restaurant
8,"Richmond, Adelaide, King",7,Coffee Shop,Café,Gym,Restaurant,Hotel,Clothing Store,Thai Restaurant,Bar,Sushi Restaurant,Bookstore,Salad Place,Pizza Place,Office,Concert Hall,Cosmetics Shop
10,"Harbourfront East, Union Station, Toronto Islands",7,Coffee Shop,Aquarium,Café,Hotel,Fried Chicken Joint,Restaurant,Scenic Lookout,Brewery,Pizza Place,Bar,Sporting Goods Shop,Bakery,Park,Italian Restaurant,Music Venue
11,"Little Portugal, Trinity",7,Bar,Coffee Shop,Men's Store,Vietnamese Restaurant,Restaurant,Café,Asian Restaurant,Brewery,Record Shop,Pizza Place,Park,Cocktail Bar,New American Restaurant,Miscellaneous Shop,Cuban Restaurant
13,"Toronto Dominion Centre, Design Exchange",7,Coffee Shop,Hotel,Restaurant,Café,Salad Place,Japanese Restaurant,American Restaurant,Seafood Restaurant,Deli / Bodega,Gastropub,Beer Bar,Asian Restaurant,Bar,Sporting Goods Shop,Steakhouse
